# Task 2
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [183]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, RBF
from sklearn.preprocessing import LabelEncoder
# Add any other imports you need here

# Data Loading
TODO: Perform data preprocessing, imputation and extract X_train, y_train and X_test
(and potentially change initialization of variables to accomodate how you deal with non-numeric data)

In [184]:
"""
This loads the training and test data, preprocesses it, removes the NaN
values and interpolates the missing data using imputation

Parameters
----------
Compute
----------
X_train: matrix of floats, training input with features
y_train: array of floats, training output with labels
X_test: matrix of floats: dim = (100, ?), test input with features
"""
# Load training data
import sklearn.impute


train_df = pd.read_csv("train.csv")
    
# print("Training data:")
# print("Shape:", train_df.shape)
# print(train_df.head(4))
# print('\n')
    
# Load test data
test_df = pd.read_csv("test.csv")

# print("Test data:")
# print(test_df.shape)
# print(test_df.head(2))

# Dummy initialization of the X_train, X_test and y_train   
# TODO: Depending on how you deal with the non-numeric data, you may want to 
# modify/ignore the initialization of these variables   
X_train = np.zeros_like(train_df.drop(['price_CHF'],axis=1))
y_train = np.zeros_like(train_df['price_CHF'])
X_test = np.zeros_like(test_df)

# TODO: Perform data preprocessing, imputation and extract X_train, y_train and X_test

train_df_CHF = train_df.dropna(subset=['price_CHF']) # Drop all rows with nan entries in "price_CHF"

X_train = train_df_CHF.drop(['price_CHF'],axis=1)
y_train = train_df_CHF['price_CHF']
X_test = test_df

spring_df = train_df_CHF[train_df_CHF['season'] == 'spring']
summer_df = train_df_CHF[train_df_CHF['season'] == 'summer']
autumn_df = train_df_CHF[train_df_CHF['season'] == 'autumn']
winter_df = train_df_CHF[train_df_CHF['season'] == 'winter']
#print(train_df_CHF)

seasons_array = [spring_df, summer_df, autumn_df, winter_df]
train_df_filled = pd.DataFrame()

for season_df in seasons_array:
    season_df_filled = pd.DataFrame(np.nan, index=season_df.index, columns=season_df.columns)
    season_df_filled.iloc[0:] = season_df.iloc[0]
    for i in range(1, 11):
        season_col_filled = season_df.iloc[:, i].fillna(np.mean(season_df.iloc[:, i]))
        season_df_filled.iloc[:, i] = season_col_filled
    
    # Append the filled season dataframe to train_df_filled
    train_df_filled = pd.concat([train_df_filled, season_df_filled], axis=0).sort_index()
#train_df_filled.drop(columns=['season'], inplace=True)

spring_df = test_df[test_df['season'] == 'spring']
summer_df = test_df[test_df['season'] == 'summer']
autumn_df = test_df[test_df['season'] == 'autumn']
winter_df = test_df[test_df['season'] == 'winter']
#print(train_df_CHF)

seasons_array = [spring_df, summer_df, autumn_df, winter_df]
test_df_filled = pd.DataFrame()

for season_df in seasons_array:
    season_df_filled = pd.DataFrame(np.nan, index=season_df.index, columns=season_df.columns)
    season_df_filled.iloc[0:] = season_df.iloc[0]
    for i in range(1, 10):
        season_col_filled = season_df.iloc[:, i].fillna(np.mean(season_df.iloc[:, i]))
        season_df_filled.iloc[:, i] = season_col_filled
    
    # Append the filled season dataframe to train_df_filled
    test_df_filled = pd.concat([test_df_filled, season_df_filled], axis=0).sort_index()
#train_df_filled.drop(columns=['season'], inplace=True)

label_encoder = LabelEncoder()
train_df_filled['season'] = label_encoder.fit_transform(train_df_filled['season'])
test_df_filled['season'] = label_encoder.fit_transform(test_df_filled['season'])

X_train = train_df_filled.drop(['price_CHF'],axis=1)
y_train = train_df_filled['price_CHF']
X_test = test_df_filled

print(X_train)
print(y_train)


assert (X_train.shape[1] == X_test.shape[1]) and (X_train.shape[0] == y_train.shape[0]) and (X_test.shape[0] == 100), "Invalid data shape"

     season  price_AUS  price_CZE  price_GER  price_ESP  price_FRA  price_UK  \
1         2  -3.421345  -3.597649  -3.675204  -2.814437  -2.440406 -2.379524   
2         0  -3.460499  -1.148417  -3.092138  -2.855250  -1.629539 -1.696917   
3         3  -2.992883  -3.372774  -2.452008  -3.533252  -1.188945 -1.682778   
4         1  -1.131171  -2.681391  -1.630410  -3.615413  -0.470111  0.658022   
5         2  -1.905878  -1.164544  -0.735178  -2.814437  -1.931793  1.294469   
..      ...        ...        ...        ...        ...        ...       ...   
894       0  -1.106242  -1.148417  -2.211310  -1.866715  -3.467347 -1.696917   
896       1  -1.612258  -1.644304  -1.967999  -1.919329  -2.094484 -3.355482   
897       2  -1.197313  -1.665509  -2.163174  -2.814437  -3.510255 -1.551595   
898       0  -1.735879  -1.781010  -2.190548  -1.788694  -1.910749 -3.516694   
899       3  -1.006364  -1.674032  -2.251319  -1.952348  -2.037431 -1.682778   

     price_ITA  price_POL  price_SVK  


/var/folders/8x/k_nyp_s16d10yl_nkhlgg7cc0000gn/T/ipykernel_70200/3250151065.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'spring' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  season_df_filled.iloc[0:] = season_df.iloc[0]
/var/folders/8x/k_nyp_s16d10yl_nkhlgg7cc0000gn/T/ipykernel_70200/3250151065.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'summer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  season_df_filled.iloc[0:] = season_df.iloc[0]
/var/folders/8x/k_nyp_s16d10yl_nkhlgg7cc0000gn/T/ipykernel_70200/3250151065.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'autumn' has dtype incompatible with float64, please explicitly cast to a c

# Modeling and Prediction
TODO: Define the model and fit it using training data. Then, use test data to make predictions

In [185]:
"""
This defines the model, fits training data and then does the prediction
with the test data 

Parameters
----------
X_train: matrix of floats, training input with 10 features
y_train: array of floats, training output
X_test: matrix of floats: dim = (100, ?), test input with 10 features

Compute
----------
y_test: array of floats: dim = (100,), predictions on test set
"""
class Model(object):
    def __init__(self):
        super().__init__()
        self._x_train = None
        self._y_train = None

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        #TODO: Define the model and fit it using (X_train, y_train)
        self.gpr = GaussianProcessRegressor(kernel=DotProduct() + RBF())
        
        self.gpr.fit(X_train, y_train)

        self._x_train = X_train
        self._y_train = y_train

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        #TODO: Use the model to make predictions y_pred using test data X_test
        y_pred=np.zeros(X_test.shape[0])
        y_pred = self.gpr.predict(X_test)
        
        assert y_pred.shape == (X_test.shape[0],), "Invalid data shape"
        return y_pred

In [186]:
model = Model()
# Use this function for training the model
model.train(X_train=X_train, y_train=y_train)
# Use this function for inferece
y_pred = model.predict(X_test)

# Saving Results
You don't have to change this

In [188]:
dt = pd.DataFrame(y_pred) 
dt.columns = ['price_CHF']
dt.to_csv('results.csv', index=False)
print("\nResults file successfully generated!")


Results file successfully generated!
